### Role Code Frequencies

So role code, role code source and qualified path (where in the ISO is it) for any Responsible Party, ie, contact, block.

So how many publishers in a data identification citation block?

In [1]:
%reload_ext autoreload
%autoreload 2

import os
import json
from lxml import etree
from semproc.rawresponse import RawResponse
from semproc.xml_utils import extract_elems
import urlparse

def convert_header_list(headers):
    return dict(
        (k.strip().lower(), v.strip()) for k, v in (
            h.split(':', 1) for h in headers)
    )

In [2]:
xp = ['//*', 'CI_ResponsibleParty', 'role', 'CI_RoleCode']

In [3]:
def generate_qualified_xpath(elem):
    tags = [elem.tag.split('}')[-1]] + [e.tag.split('}')[-1] for e in elem.iterancestors()]
    index = tags.index('CI_ResponsibleParty')
    tags = tags[index+1:]
    tags.reverse()
    return '/'.join(tags)

In [5]:
# i am going to cheat a bit here. just look
# at the responses that included an iso standard and version
# element. if it quacks like a duck....

'''
<gmd:role>
    <gmd:CI_RoleCode 
        codeList="http://www.ngdc.noaa.gov/metadata/published/xsd/schema/resources/Codelist/gmxCodelists.xml#CI_RoleCode" 
        codeListValue="pointOfContact">pointOfContact</gmd:CI_RoleCode>
</gmd:role>
'''

with open('outputs/iso_versions_by_response.txt', 'r') as f:
    responses = f.readlines()
responses = [d.split(', Standard: ')[0].replace('File: ', '').strip() for d in responses[1:] if d]

ROLE_PATH = 'outputs/rolecodes.txt'
with open(ROLE_PATH, 'w') as f:
    f.write('file,codelist,codelist_value,xpath\n')

for r in responses:
    with open(os.path.join('/Users/sparky/Documents/solr_responses/solr_20150922_docs/', r), 'r') as g:
        data = json.loads(g.read())
    
    response = data.get('raw_content')
    headers = convert_header_list(data.get('response_headers', []))
    content_type = headers.get('content-type', '')
    
    rr = RawResponse(response, content_type)
    try:
        content = rr.clean_raw_content()
    except:
        print 'FAILED ', f
        continue

    if rr.datatype != 'xml':
        continue
    
    try:
        xml = etree.fromstring(content)
    except:
        continue
    
    # let's sort out the role codes
    for role_elem in extract_elems(xml, xp):
        codelist = role_elem.attrib.get('codeList', 'NONE').strip()
        codelist_value = role_elem.attrib.get('codeListValue', 'NONE').strip()
        
        # generate the path
        tags = generate_qualified_xpath(role_elem)
        tags = tags.strip() if tags else 'NONE'
        
        with open(ROLE_PATH, 'a') as g:
            g.write(','.join([r.replace('.json', ''), codelist, codelist_value, tags]) +'\n')
        
        
        

In [4]:
# run the same with the host included
with open('outputs/iso_versions_by_response.txt', 'r') as f:
    responses = f.readlines()
responses = [d.split(', Standard: ')[0].replace('File: ', '').strip() for d in responses[1:] if d]

ROLE_PATH = 'outputs/rolecodes_by_host.txt'
with open(ROLE_PATH, 'w') as f:
    f.write('file,host,codelist,codelist_value,xpath\n')

for r in responses:
    with open(os.path.join('/Users/sparky/Documents/solr_responses/solr_20150922_docs/', r), 'r') as g:
        data = json.loads(g.read())
    
    response = data.get('raw_content')
    headers = convert_header_list(data.get('response_headers', []))
    content_type = headers.get('content-type', '')
    url = data.get('url')
    host = urlparse.urlparse(url).netloc
    
    rr = RawResponse(response, content_type)
    try:
        content = rr.clean_raw_content()
    except:
        print 'FAILED ', f
        continue

    if rr.datatype != 'xml':
        continue
    
    try:
        xml = etree.fromstring(content)
    except:
        continue
    
    # let's sort out the role codes
    for role_elem in extract_elems(xml, xp):
        codelist = role_elem.attrib.get('codeList', 'NONE').strip()
        codelist_value = role_elem.attrib.get('codeListValue', 'NONE').strip()
        
        # generate the path
        tags = generate_qualified_xpath(role_elem)
        tags = tags.strip() if tags else 'NONE'
        
        with open(ROLE_PATH, 'a') as g:
            g.write(','.join([r.replace('.json', ''), host, codelist, codelist_value, tags]) +'\n')
        

### Schema Location Verification

Is it a local reference?

Is it linkrot?

In [12]:
import requests
import csv

responses = []

with open('outputs/rolecodes_codelists.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        codelist = row[0].strip()
        
        if not codelist.startswith('http'):
            responses.append('{0},not HTTP'.format(codelist))
            continue

        try:
            rsp = requests.get(codelist)
        except:
            responses.append('{0},Exception'.format(codelist))
            continue
        
        responses.append('{0},{1}'.format(codelist, rsp.status_code))
        
# forgot about the header today
responses = responses[1:]
responses

['./resources/codeList.xml#CI_RoleCode,not HTTP',
 'CI_RoleCode,not HTTP',
 'file://Agi-s000001/afdelingen/GSMS/Extern/Applicaties_en_Tools/Databeheer/Metadata/GeoMDE_22/resources/customCodelists_new.xml#CI_RoleCode,not HTTP',
 'file://Shr-ipvw-gpr001/metadatatools/Metadatamaker/config_bestanden/customCodelists.xml#CI_RoleCode,not HTTP',
 'file://appconf.intranet.rws.nl/Appconf/metadatatools/Metadatamaker/config_bestanden/customCodelists.xml#CI_RoleCode,not HTTP',
 'http://adl.brs.gov.au/anrdl/resources/codeList/codeList20120313.xml#CI_RoleCode,Exception',
 'http://asdd.ga.gov.au/asdd/profileinfo/gmxCodelists.xml#CI_RoleCode,200',
 'http://aws2.caris.com/sfs/schemas/iso/19139/20070417/resources/Codelist/gmxCodelists.xml#CI_RoleCode,200',
 'http://data.daff.gov.au/anrdl/resources/codeList/codeList20120313.xml#CI_RoleCode,200',
 'http://mdtranslator.adiwg.org/api/codelists?format=xml#CI_RoleCode,200',
 'http://nap.geogratis.gc.ca/metadata/register/napMetadataRegister.xml#IC_90,200',
 'ht

In [14]:
len([r for r in responses if ',40' in r])

7

7 of 33 file-based references.

18 of 33 returned 200s.

7 of 33 are 404s.

1 failed request.


### Extracting any code list reference

633,590 references. See the postgres for details.

In [17]:
# run the same with the host included
# xp = ['//*', '@codeList']

def generate_qualified_xpath(elem):
    tags = [elem.tag.split('}')[-1]] + [e.tag.split('}')[-1] for e in elem.iterancestors()]
    tags.reverse()
    return '/'.join(tags)

with open('outputs/iso_versions_by_response.txt', 'r') as f:
    responses = f.readlines()
responses = [d.split(', Standard: ')[0].replace('File: ', '').strip() for d in responses[1:] if d]

ROLE_PATH = 'outputs/all_codelists_by_host.txt'
with open(ROLE_PATH, 'w') as f:
    f.write('file,host,codelist,xpath\n')

for r in responses:
    with open(os.path.join('/Users/sparky/Documents/solr_responses/solr_20150922_docs/', r), 'r') as g:
        data = json.loads(g.read())
    
    response = data.get('raw_content')
    headers = convert_header_list(data.get('response_headers', []))
    content_type = headers.get('content-type', '')
    url = data.get('url')
    host = urlparse.urlparse(url).netloc
    
    rr = RawResponse(response, content_type)
    try:
        content = rr.clean_raw_content()
    except:
        print 'FAILED ', f
        continue

    if rr.datatype != 'xml':
        continue
    
    try:
        xml = etree.fromstring(content)
    except:
        continue
    
    # let's sort out the role codes
    for role_elem in xml.xpath('//*[@codeList]'):
        codelist = role_elem.attrib.get('codeList', 'NONE').strip()
        
        # generate the path
        tags = generate_qualified_xpath(role_elem)
        tags = tags.strip() if tags else 'NONE'
        
        with open(ROLE_PATH, 'a') as g:
            g.write(','.join([r.replace('.json', ''), host, codelist, tags]) +'\n')